In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
import json
with open('/kaggle/input/datasetSecure/data.json', 'r') as file:
    datasec = json.load(file)

In [ ]:
token=datasec["token"]
command = f"huggingface-cli login --token {token}"

In [ ]:
!{command}

In [ ]:
!pip install langchain_community
!pip install sentencepiece

In [ ]:
!pip install torch -U
!pip install transformers accelerate git+https://github.com/kashif/diffusers.git@a3dc21385b7386beb3dab3a9845962ede6765887

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from transformers import pipeline
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
model1 = ""
# model = "soundarya2873/llama-2-7b-chat-finetuned1"
# model="meta-llama/Llama-2-7b-chat-hf"
model="alapatirohith/llama-2-7b-chat-finetuned1_"
# model="Dhanunjaysuppa/llama-2-7b-chat-finetuned1_fun"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
torch.cuda.empty_cache()

In [ ]:
id = "6617f01ba634747a22e3ab3a"
text = """KESHAV MEMORIAL INSTITUTE OF TECHNOLOGY
(AN AUTONOMOUS INSTITUTION)
Narayanguda, Hyderabad, Telangana 500029
B.Tech 2 Year 1 Sem KR21 Regular MARCH-2023 Results
Hall Ticket: Enter HallTicket Number
Date of
Birth:DD/MM/YYYY
Show Results      Clear
KESHAV MEMORIAL INSTITUTE OF TECHNOLOGY
(AN AUTONOMOUS INSTITUTE)
Narayanguda, Hyderabad, Telangana 500029
B.Tech 2 Year 1 Sem KR21 Regular MARCH-2023 Result
Name SUPPA DHANUNJAYA Hall Ticket 21BD1A1255 Branch IT
S.NO Subject Code SubjectsGrade
SecuredGrade
Points ResultCredits
1 21301 Machine Learning Using Python Lab O 10 P 1
2 21302 Data Structures Through C++ Lab O 10 P 2
3 21303 Analog and Digital Electronics Lab A+ 9 P 1
4 21304 Gender Sensitization Lab O 10 P 0
5 213AA Mathematical Foundations of Computer
ScienceA 8 P 4
6 213AB Computer Organization and Architecture B 6 P 3
7 213AC Data Structures Through C++ F 0 F 0
8 213AD Analog and Digital Electronics B 6 P 3
9 213AE Introduction to Machine Learning B 6 P 4
*Note: -2 Indicates
WithheldREGISTERED : 9 APPEARED : 9 PASSED : 8 TOTAL 18
Result : FAIL Semester Grade Point Average(SGPA) : -Print
Copyright � 2023 Teleparadigm Networks
"""

In [ ]:
import math
num_clusters=math.ceil(len(text)/1200)

In [ ]:
num_clusters

In [ ]:
# l=text.split(". ")
import re
l=re.split(r'\. |\n',text)
l

In [ ]:
i=0
while i<len(l):
  if len(l[i])<=1:
    l.remove(l[i])
  else:
    i=i+1

In [ ]:
l

In [ ]:
!pip install textdistance

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import textdistance  # Make sure to install this library using: pip install textdistance

# Sample string l
# l = ["apple", "banana", "orange", "grape", "cherry", "pineapple", "blueberry", "strawberry"]

# Function to calculate Levenshtein distance between two strings
def levenshtein_distance(str1, str2):
    return textdistance.levenshtein.normalized_distance(str1, str2)

# Convert string l to a matrix of token counts
# vectorizer = TfidfVectorizer(analyzer='char', use_idf=False)
# X = vectorizer.fit_transform(l).toarray()

# Calculate pairwise Levenshtein distances
distances = np.zeros((len(l), len(l)))
for i in range(len(l)):
    for j in range(i+1, len(l)):
        distances[i, j] = distances[j, i] = levenshtein_distance(l[i], l[j])

# Perform k-means clustering
# num_clusters = 6  # You can adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(distances)

# Add cluster labels to the original l
cluster_labels = kmeans.labels_
# result = pd.DataFrame({'String': l, 'Cluster': cluster_labels})
# m=max(cluster_labels)+1
# d={}
# d2={}
# for i in range(len(cluster_labels)):
#   x=d.get(cluster_labels[i],"")+l[i]+". "
#   if(len(x)>1200):
#     if(cluster_labels[i] in d2):
#       d[d2[cluster_labels[i]]]=d[d2[cluster_labels[i]]]+l[i]+". "
#     else:
#       d2[cluster_labels[i]]=m
#       d[m]=l[i]+". "
#       m+=1
#   else:
#     d[cluster_labels[i]]=x

d={}
for i in range(len(cluster_labels)):
  d[cluster_labels[i]]=d.get(cluster_labels[i],"")+l[i]+". "

# Display the result
# print(result)
l2=list(d.values())
def split_long_strings(strings, max_length):
    result = []
    for string in strings:
        if len(string) > max_length:
            # Split the long string into parts of max_length
            # parts = [string[i:i+max_length] for i in range(0, len(string), max_length)]
            strs=string.split(". ")
            str1=strs[0]
            for i in range(1,len(strs)):
                if(len(str1+strs[i])<=max_length):
                    str1+=strs[i]
                else:
                    result.append(str1)
                    str1=strs[i]
            result.append(str1)
        else:
            result.append(string)
    return result

# Example usage:
cluster_list = split_long_strings(l2,1200)
cluster_list

In [ ]:
from langchain import PromptTemplate,  LLMChain
def generate_summary(text_chunk):
    # Defining the template to generate summary
    template = """
    Context:
    You are tasked with summarizing a large body of text that has been divided into clusters for efficient processing. Each cluster contains related information, and the goal is to generate a concise and informative summary for each cluster.
    Input text:
    ```{text}```
    SUMMARY:
    """
    prompt = PromptTemplate(template=template, input_variables=["text"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    summary = llm_chain.run(text_chunk)
    torch.cuda.empty_cache()
    return summary

In [ ]:
# !pip install pymongo
!/opt/conda/bin/python3.10 -m pip install "pymongo[srv]"

In [ ]:
from pymongo import MongoClient
from bson import ObjectId
connection_string=datasec["data"]
client = MongoClient(connection_string)
db = client.Llama2
collection = db.summary
query_criteria = {"_id": ObjectId(id)}

In [ ]:
import torch
# chunk_summaries = []
torch.cuda.empty_cache()
import os

combined_summary=""
# Set the max_split_size_mb environment variable
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:50'
for chunk in cluster_list:
    summary = generate_summary(chunk)
    torch.cuda.empty_cache()
    print(summary)
    torch.cuda.empty_cache()
    print("done")
    torch.cuda.empty_cache()
    # Clear CUDA memory after each iteration
    combined_summary+=summary+"\n"
    update_query = {"$set": {"summary": combined_summary}}
    collection.update_one(query_criteria, update_query)
    # chunk_summaries.append(summary)


# combined_summary = "\n".join(chunk_summaries)
client.close()

In [ ]:
print(combined_summary)